### Importing libraries

In [1]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,confusion_matrix, classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

### Reading all raw data

In [4]:
train = pd.read_csv(r'C:\Users\91902\Desktop\edavancer  submite project\project no 6\train.csv')
test = pd.read_csv(r'C:\Users\91902\Desktop\edavancer  submite project\project no 6\test_share.csv')
med = pd.read_csv(r'C:\Users\91902\Desktop\edavancer  submite project\project no 6\medical_history.csv')
demo = pd.read_csv(r'C:\Users\91902\Desktop\edavancer  submite project\project no 6\demographic_details.csv')

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
# Before merging check the data type, always use int or string data type for primary keys
# Convert PatientId from Float to Integer
train['PatientId'] = train['PatientId'].astype('int64')
test['PatientId'] = test['PatientId'].astype('int64')
med['PatientId'] = med['PatientId'].astype('int64')
demo['PatientId'] = demo['PatientId'].astype('int64')

In [7]:
train.shape, test.shape, med.shape, demo.shape

((99490, 6), (10854, 5), (62299, 5), (62299, 5))

In [8]:
#merging medical history & demographic details 
med_demo = pd.merge(med, demo, how ='outer', on ='PatientId')

In [9]:
med_demo.shape

(62299, 9)

In [10]:
#merging above created data with train & test dataset
train_md = pd.merge(train, med_demo, how ='left', on ='PatientId')
test_md = pd.merge(test, med_demo, how ='left', on ='PatientId')

In [11]:
train_md.shape, test_md.shape

((99490, 14), (10854, 13))

In [12]:
train_md.isnull().sum(), test_md.isnull().sum()

(PatientId         0
 AppointmentID     0
 ScheduledDay      0
 AppointmentDay    0
 SMS_received      0
 No-show           0
 Hipertension      0
 Diabetes          0
 Alcoholism        0
 Handcap           0
 Gender            0
 Age               0
 Neighbourhood     0
 Scholarship       0
 dtype: int64,
 PatientId         0
 AppointmentID     0
 ScheduledDay      0
 AppointmentDay    0
 SMS_received      0
 Hipertension      0
 Diabetes          0
 Alcoholism        0
 Handcap           0
 Gender            0
 Age               0
 Neighbourhood     0
 Scholarship       0
 dtype: int64)

In [13]:
train_md.info(), test_md.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99490 entries, 0 to 99489
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PatientId       99490 non-null  int64 
 1   AppointmentID   99490 non-null  int64 
 2   ScheduledDay    99490 non-null  object
 3   AppointmentDay  99490 non-null  object
 4   SMS_received    99490 non-null  int64 
 5   No-show         99490 non-null  object
 6   Hipertension    99490 non-null  int64 
 7   Diabetes        99490 non-null  int64 
 8   Alcoholism      99490 non-null  int64 
 9   Handcap         99490 non-null  int64 
 10  Gender          99490 non-null  object
 11  Age             99490 non-null  int64 
 12  Neighbourhood   99490 non-null  object
 13  Scholarship     99490 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 11.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10854 entries, 0 to 10853
Data columns (total 13 columns):
 #   Column          Non-Nu

(None, None)

In [14]:
# rename the columns
train_md = train_md.rename(columns={'Hipertension': 'Hypertension', 'Handcap': 'Handicap', 'No-show': 'No_Show'})
test_md = test_md.rename(columns={'Hipertension': 'Hypertension', 'Handcap': 'Handicap'})

In [15]:
# Adding new columns in baith dataframe
train_md['Data'] = 'train'
test_md['Data'] = 'test'

In [16]:
# moving No_Show column to end
column_to_move = train_md.pop("No_Show")

# insert column with insert(location, column_name, column_value)
train_md.insert(14, "No_Show", column_to_move)

# different method
#df1 = df.pop('No_Show') # remove column No_Show and store it in df1
#df['No_Show']=df1 # add No_Show series as a 'new' column.

In [17]:
train_md.columns, test_md.columns

(Index(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay',
        'SMS_received', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap',
        'Gender', 'Age', 'Neighbourhood', 'Scholarship', 'Data', 'No_Show'],
       dtype='object'),
 Index(['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay',
        'SMS_received', 'Hypertension', 'Diabetes', 'Alcoholism', 'Handicap',
        'Gender', 'Age', 'Neighbourhood', 'Scholarship', 'Data'],
       dtype='object'))

In [18]:
#To make both dataframe in same order
#test_md = test_md[train_md.columns]

In [19]:
train_md.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,Hypertension,Diabetes,Alcoholism,Handicap,Gender,Age,Neighbourhood,Scholarship,Data,No_Show
0,29872499824296,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,1,0,0,0,F,62,JARDIM DA PENHA,0,train,No
1,558997776694438,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,0,0,0,0,M,56,JARDIM DA PENHA,0,train,No
2,867951213174,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,0,0,0,0,F,8,PONTAL DE CAMBURI,0,train,No
3,8841186448183,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,1,1,0,0,F,56,JARDIM DA PENHA,0,train,No
4,95985133231274,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,1,0,0,0,F,76,REPÚBLICA,0,train,No


In [20]:
test_md.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,Hypertension,Diabetes,Alcoholism,Handicap,Gender,Age,Neighbourhood,Scholarship,Data
0,7223289184215,5640433,2016-04-29T10:43:14Z,2016-04-29T00:00:00Z,0,0,0,0,0,F,46,DA PENHA,0,test
1,863229818887631,5616091,2016-04-25T13:29:16Z,2016-04-29T00:00:00Z,1,0,0,0,0,M,13,CONQUISTA,0,test
2,443858731994526,5637909,2016-04-29T07:30:31Z,2016-04-29T00:00:00Z,0,0,0,0,0,M,6,CONQUISTA,0,test
3,41799315536436,5638018,2016-04-29T07:37:45Z,2016-04-29T00:00:00Z,0,0,0,0,0,M,2,NOVA PALESTINA,0,test
4,36477615234971,5614045,2016-04-25T10:01:13Z,2016-04-29T00:00:00Z,1,0,0,0,0,F,3,CONQUISTA,1,test


In [21]:
# Concat train and test data-
df = pd.concat([train_md,test_md], axis=0)

In [22]:
df.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,Hypertension,Diabetes,Alcoholism,Handicap,Gender,Age,Neighbourhood,Scholarship,Data,No_Show
0,29872499824296,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,1,0,0,0,F,62,JARDIM DA PENHA,0,train,No
1,558997776694438,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,0,0,0,0,M,56,JARDIM DA PENHA,0,train,No
2,867951213174,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,0,0,0,0,F,8,PONTAL DE CAMBURI,0,train,No
3,8841186448183,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,1,1,0,0,F,56,JARDIM DA PENHA,0,train,No
4,95985133231274,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,1,0,0,0,F,76,REPÚBLICA,0,train,No


In [23]:
df.isnull().sum()

PatientId             0
AppointmentID         0
ScheduledDay          0
AppointmentDay        0
SMS_received          0
Hypertension          0
Diabetes              0
Alcoholism            0
Handicap              0
Gender                0
Age                   0
Neighbourhood         0
Scholarship           0
Data                  0
No_Show           10854
dtype: int64

In [24]:
# converting object type to datetime
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.date.astype('datetime64[ns]')

df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.date.astype('datetime64[ns]')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110344 entries, 0 to 10853
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   PatientId       110344 non-null  int64         
 1   AppointmentID   110344 non-null  int64         
 2   ScheduledDay    110344 non-null  datetime64[ns]
 3   AppointmentDay  110344 non-null  datetime64[ns]
 4   SMS_received    110344 non-null  int64         
 5   Hypertension    110344 non-null  int64         
 6   Diabetes        110344 non-null  int64         
 7   Alcoholism      110344 non-null  int64         
 8   Handicap        110344 non-null  int64         
 9   Gender          110344 non-null  object        
 10  Age             110344 non-null  int64         
 11  Neighbourhood   110344 non-null  object        
 12  Scholarship     110344 non-null  int64         
 13  Data            110344 non-null  object        
 14  No_Show         99490 non-null   obje

### No_Show

In [26]:
df['No_Show'] = np.where(df['No_Show'] == 'Yes', 1, 0)
df['No_Show'].unique()

array([0, 1])

### Handicap

In [27]:
df['Handicap'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [28]:
# change all the Handicap set above 0 as 1
df['Handicap'] = np.where(df['Handicap'] >= 1, 1, 0)

# different method
# train_md.loc[train_md.Handicap > 0, 'Handicap'] = 1

In [29]:
df['Handicap'].unique()

array([0, 1])

### Gender

In [30]:
df['Gender'] = np.where(df['Gender'] == "F", 1, 0)

In [31]:
df['Gender'].unique()

array([1, 0])

### Waiting_Day

In [32]:
# Adding a column which gives total waiting days
df['Waiting_Day'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

In [33]:
# drop unwanted columns
df.drop(['AppointmentDay','ScheduledDay'],axis=1,inplace=True)

In [34]:
df['Waiting_Day'].unique()

array([  0,   2,   3,   1,   4,   9,  29,  10,  23,  11,  18,  17,  14,
        28,  24,  21,  15,  16,  22,  43,  30,  31,  42,  56,  45,  46,
        39,  37,  32,  38,  44,  50,  52,  53,  65,  67,  91,  66,  78,
        87, 115, 109,  63,  70,  72,  84,  57,  58,  51,  59,  41,  73,
        64,  49,  20,  33,  34,   6,  35,  36,  12,  13,  40,  47,   8,
         5,   7,  25,  26,  48,  27,  19,  61,  55,  62, 176,  54,  77,
        69,  83,  76,  89,  81,  60,  68,  75,  85, 112,  79,  -1,  80,
        86,  98,  94, 142, 155, 162, 169, 104, 133, 125,  96,  88,  90,
       151, 126, 127, 111, 119,  74,  71,  82, 103, 108, 110, 102, 122,
       101, 105,  92,  97,  93, 107,  95,  -6, 139, 132, 179, 117, 146],
      dtype=int64)

In [35]:
# Waiting_Day cannot be less than 0
df[(df['Waiting_Day'] < 0)].Waiting_Day.value_counts()

-1    4
-6    1
Name: Waiting_Day, dtype: int64

In [36]:
df['Waiting_Day'][df['Waiting_Day'] < 0] = df['Waiting_Day'].mean()

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110344 entries, 0 to 10853
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   PatientId      110344 non-null  int64  
 1   AppointmentID  110344 non-null  int64  
 2   SMS_received   110344 non-null  int64  
 3   Hypertension   110344 non-null  int64  
 4   Diabetes       110344 non-null  int64  
 5   Alcoholism     110344 non-null  int64  
 6   Handicap       110344 non-null  int32  
 7   Gender         110344 non-null  int32  
 8   Age            110344 non-null  int64  
 9   Neighbourhood  110344 non-null  object 
 10  Scholarship    110344 non-null  int64  
 11  Data           110344 non-null  object 
 12  No_Show        110344 non-null  int32  
 13  Waiting_Day    110344 non-null  float64
dtypes: float64(1), int32(3), int64(8), object(2)
memory usage: 11.4+ MB


### Age

In [38]:
df['Age'].unique()

array([ 62,  56,   8,  76,  23,  39,  21,  19,  30,  29,  22,  28,  54,
        15,  50,  40,  46,   4,  13,  65,  45,  51,  32,  12,  38,  18,
        63,  64,  85,  59,  55,  71,  49,  78,  31,  58,  27,   6,   2,
        11,   7,   0,   3,   1,  69,  61,  60,  67,  68,  36,  10,  35,
        20,  26,  34,  33,  42,   5,  47,  17,  16,  41,  44,  37,  24,
        66,  77,  81,  70,  53,  75,  73,  52,  74,  43,  89,  57,  14,
         9,  48,  79,  72,  80,  87,  25,  83,  88,  84,  82,  90,  94,
        86,  91,  98,  92,  96,  93,  95,  97, 102, 115, 100,  99,  -1],
      dtype=int64)

In [39]:
# Age cannot be less than 0
df['Age'][df['Age']<0] = df.Age.mean()

In [40]:
# Creating dummies
dum = df['Neighbourhood'].value_counts(normalize=True)
np.sum(dum > .02)

18

In [41]:
## So only 18 dummies as per above logic
df_dum = dum[dum > .02].index ## these are the ones which we should use as dummies
df = df.assign(**{
                "new_nhood": np.where(df.Neighbourhood.isin(df_dum), df.Neighbourhood, 'Other') 
                 }).drop('Neighbourhood', axis=1)

In [42]:
# concat to df and the drop unwanted column
df = pd.concat([df, pd.get_dummies(df.new_nhood, drop_first = True)],axis=1)
df.drop(['new_nhood'],axis=1,inplace=True)

In [43]:
# split the data into train and test as previous
train_df =df[df['Data'] =='train']
test_df = df[df['Data'] =='test']

In [44]:
# Drop unwanted columns
train_df.drop('Data',axis=1,inplace=True)
test_df.drop('Data',axis=1,inplace=True)
test_df.drop('No_Show',axis=1,inplace=True)

In [45]:
test_df.columns

Index(['PatientId', 'AppointmentID', 'SMS_received', 'Hypertension',
       'Diabetes', 'Alcoholism', 'Handicap', 'Gender', 'Age', 'Scholarship',
       'Waiting_Day', 'CARATOÍRA', 'CENTRO', 'DA PENHA', 'ILHA DO PRÍNCIPE',
       'ITARARÉ', 'JABOUR', 'JARDIM CAMBURI', 'JARDIM DA PENHA',
       'JESUS DE NAZARETH', 'MARIA ORTIZ', 'NOVA PALESTINA', 'Other',
       'RESISTÊNCIA', 'SANTA MARTHA', 'SANTO ANDRÉ', 'SANTO ANTÔNIO',
       'SÃO PEDRO', 'TABUAZEIRO'],
      dtype='object')

In [46]:
train_df.shape, test_df.shape

((99490, 30), (10854, 29))

In [47]:
# check if header's are consistent in both train and test
set(train_df.columns) - set(test_df.columns)

{'No_Show'}

In [48]:
# Split the train dataset into features and target variable
X = train_df.drop('No_Show', axis=1)
y = train_df['No_Show']

In [49]:
# Standardising the data
scaler = StandardScaler()
train = scaler.fit_transform(X)
# here we just need to standardise the train data, no need to do this on target

In [50]:
# checking event rate
y.value_counts()

0    79360
1    20130
Name: No_Show, dtype: int64

In [51]:
# Balancing the data
ros = RandomOverSampler(random_state=2)
X_resample,y_resample = ros.fit_resample(X,y)
from collections import Counter
print(sorted(Counter(y_resample).items()),y_resample.shape)

[(0, 79360), (1, 79360)] (158720,)


In [52]:
# Split the train dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.2, random_state=5)

In [53]:
# Train the model
xgb = XGBClassifier()

In [54]:
# fitting the model
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [55]:
# Predict the target variable for the validation set
y_pred = xgb.predict(X_test)

In [56]:
# Calculate the roc_auc score for the validation set
score = roc_auc_score(y_test, y_pred)

print("roc_auc score: ", score)

roc_auc score:  0.7217678495158975


In [58]:
# Predict the target variable for the test set
test_pred = xgb.predict(test_df)

In [59]:
# Create a submission file with the predicted probabilities
submission_df = pd.DataFrame({'PatientId': test_df['PatientId'], 'No_Show': test_pred})
submission_df.to_csv('No_Shows_for_Medical_Appointments_08.csv', index=False)